<a href="https://colab.research.google.com/github/Marcelle-git/DeepLearningTask/blob/main/Copie_de_DeepLearning_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' ## hide tf warnings
import tensorflow as tf
import matplotlib.pyplot as plt

Access to my drive because I imported the image dataset in my Google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import the zipfile library for extracting the files that I imported on Drive and copy the paths for loading images

In [ ]:
#import zipfile


In [ ]:
#zip_train = zipfile.ZipFile('/content/drive/MyDrive/archive.zip', 'r')
#zip_train.extractall('/content/drive/MyDrive/Colab Notebooks')
#zip_train.close()

Loading image dataset train and test

In [3]:
train_dataset_path = '/content/drive/MyDrive/Colab Notebooks/seg_train/seg_train'
test_dataset_path = '/content/drive/MyDrive/Colab Notebooks/seg_test/seg_test'

In [4]:
IMG_WIDTH = 150       #Here are the the size of the images
IMG_HEIGHT = 150
BATCH_SIZE = 32       #I define the batch size

In [5]:
print('Training images:')
train_generator = tf.keras.preprocessing.image_dataset_from_directory(train_dataset_path,
                                              image_size=(IMG_WIDTH, IMG_HEIGHT),
                                              batch_size=BATCH_SIZE,
                                              label_mode="categorical")

print('Testing images:')
test_generator = tf.keras.preprocessing.image_dataset_from_directory(test_dataset_path,
                                              image_size=(IMG_WIDTH, IMG_HEIGHT),
                                              batch_size=BATCH_SIZE,
                                              label_mode="categorical")

Training images:
Found 14034 files belonging to 6 classes.
Testing images:
Found 3000 files belonging to 6 classes.


In [8]:
# Checkpoint path
CheckpointPath="EfficientNetB0.h5"

CheckpointCallback= tf.keras.callbacks.ModelCheckpoint(
                            filepath=CheckpointPath,
                            verbose=1,
                            save_best_only=True,
                            save_weights_only=False,
                            monitor='val_accuracy',
                            save_freq=1
)

In [9]:
Learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=5,
                                            verbose=1,
                                            factor=0.5,
                                             mode='auto',
                                            cooldown=1 )

In [10]:
Early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                  min_delta=0.001,
                                                  patience = 5,
                                                  verbose = 1,
                                                  restore_best_weights=True)

In [11]:
# Create base model (load the pre-trained model)
base_model_EfficientNetB0 = tf.keras.applications.EfficientNetB0(include_top=False)

# Freeze the base model (pre-trained patterns aren't update during training)
base_model_EfficientNetB0.trainable = False

# Create inputs
inputs = tf.keras.layers.Input(shape=(150,150,3), name="input_layer")

# Pass the inputs to the base model
x = base_model_EfficientNetB0(inputs)

# Average pooling the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

# Create output activation layer
outputs = tf.keras.layers.Dense(6,activation="softmax", name="output_layer")(x)

# Combine the inputs with the outputs into a model
model_1 = tf.keras.Model(inputs, outputs)

# Compile the model
model_1.compile(loss="categorical_crossentropy",
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
               metrics=["accuracy"])

# Fit the model and save its history
history_1 = model_1.fit(train_generator,
                       epochs=5,
                       steps_per_epoch=len(train_generator),
                       validation_data=test_generator,
                       validation_steps=len(test_generator),
                       callbacks=[
                                  CheckpointCallback,
                                  Early_stopping,
                                  Learning_rate_reduction])

16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/5


  1/439 [..............................] - ETA: 10:47:46 - loss: 1.9035 - accuracy: 0.1562

  2/439 [..............................] - ETA: 25:07 - loss: 1.8638 - accuracy: 0.1562   

  3/439 [..............................] - ETA: 23:46 - loss: 1.9037 - accuracy: 0.1250

  4/439 [..............................] - ETA: 23:50 - loss: 1.8967 - accuracy: 0.1094

...

435/439 [============================>.] - ETA: 4s - loss: 0.2692 - accuracy: 0.9103

436/439 [============================>.] - ETA: 3s - loss: 0.2691 - accuracy: 0.9104

437/439 [============================>.] - ETA: 2s - loss: 0.2692 - accuracy: 0.9104

438/439 [============================>.] - ETA: 1s - loss: 0.2689 - accuracy: 0.9105

439/439 [==============================] - 600s 1s/step - loss: 0.2688 - accuracy: 0.9106 - val_loss: 0.2761 - val_accuracy: 0.9040 - lr: 1.0000e-04
